In [1]:
import pandas as pd
import numpy as np
import yaml
from sklearn.metrics import mean_squared_error
from C_functions_opti import get_LHS_samples, model_optimization, plot_estimation

##### 1. Load all data that is needed

In [2]:
# Load experimental data
df_exp = pd.read_csv('data/data_combined.csv')
biomass_exp = df_exp['Biomass [g/L]']
substrate_exp = df_exp['Glucose [g/L]']

In [3]:
# Load parameters from YAML file
with open('config/parameters.yml', 'r') as file:
    param = yaml.safe_load(file)

##### 2. Define all possible equations for mu and qs

The growth rate and the substrate uptake rate are dependend of each other - meaning that if we define a equation for **mu**, then **qs** is defined by mu with $qs=mu/Yxs$. Contrarily, when **qs** is defined by an ODE, then **mu** is calculated with $mu = qs * Yxs$.

In [4]:
mu0 = lambda mu_max, c_glucose, Ks: mu_max * c_glucose / (c_glucose + Ks) # -- MONOD
mu1 = lambda mu_max, c_biomass, X_max: mu_max * (1 - (c_biomass/ X_max)) # -- LOGISTIC
mu2 = lambda mu_max, c_glucose, Ks, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC
mu3 = lambda mu_max, c_glucose, Ks, Ki, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC + INHIBITION
mu4 = lambda qs, Yxs: qs * Yxs # for changing qs

In [5]:
qs0 = lambda qs_max, c_glucose, Ks_qs: qs_max * c_glucose / (Ks_qs + c_glucose) # -- MONOD
qs1 = lambda qs_max, c_glucose, Ks_qs, Ki, glu_met: qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met)) # -- MONOD + NON COMPETITIVE INHIBITION
qs2 = lambda qs_max, c_glucose, Ks_qs, c_biomass, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(c_biomass * lag))) # -- MONOD + METABOLIZED GLU
qs3 = lambda mu, Yxs: mu / Yxs # for changing mu

In [6]:
mu_all=[mu0, mu1, mu2, mu3, mu4]
qs_all=[qs0, qs1, qs2, qs3]

##### 3. Define functions that are needed

In [7]:
# Root mean squared error is the objective function
def objective_function(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

##### 4. Define set of parameters in config file

In [8]:
## not needed anymore
# Extract the parameter combination and the number of the set in order to save it in the correct folder
est_mu_max = param['est_mu_max'] # ['set0']
est_Ks = param['est_ks']

In [9]:
# Set the number of samples and parameters
num_samples = 50
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.01, 0.6],    # Range for parameter 1 mu_max
    [18, 25],       # Range for parameter 2 X_max
    [0.1, 10.0],    # Range for parameter 3 - Ks
    [0.1, 10.0],    # Range for parameter 4 - Ks_qs
    [6.0, 8.0],     # Range for parameter 5 - Ki
    [0.2, 0.6],     # Range for parameter 6 - Yxs
    [0.5, 1.5],     # Range for parameter 7 - qs_max
    [0.05, 0.5],   # Range for parameter 8 - m_s
    [0.001, 0.02],  # Range for parameter 9 - lag
]

In [10]:
LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(50, 9)

##### 5. Run it

In [11]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    for i in range(len(mu_all)):
        # Define the combination of equations
        mu_eq = mu_all[i]; num_mu = i

        # mu 0 to mu3 are combined with qs3 while for mu4 all qs except for qs3 are combined
        if i != 4:
            j = 3
            qs_eq = qs_all[j]; num_qs = j   

            # Make the predictions and calculate the error
            rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
            # save the parameters in a dataframe
            append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
            rmse_one_set.append(append_list)

            # Make a plot and save it
            title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
            plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

            if rmse <= float(5):
                plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)
        
        elif i == 4:
            for j in range(3):
                qs_eq = qs_all[j]; num_qs = j   

                # Make the predictions and calculate the error
                rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
                # save the parameters in a dataframe
                append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
                rmse_one_set.append(append_list)

                # Make a plot and save it
                title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
                plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

                if rmse <= float(10):
                    plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)

    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/new_eq/data.csv')

df_all_sets.head(10)

,set,mu,qs,mu_max,X_max,Ks,Ks_qs,Ki,Yxs,qs_max,m_s,lag,rmse
337,48,1,3,0.182944,22.545783,3.060362,6.921956,6.578043,0.425618,0.971633,0.417952,0.013150,3.279
44,6,2,3,0.541981,18.037981,7.580407,1.238825,7.022540,0.350874,0.851657,0.373909,0.015245,4.078
64,9,1,3,0.322665,19.245329,5.521061,5.694683,6.359801,0.463613,0.800436,0.311880,0.001825,4.156
296,42,2,3,0.346008,18.947696,5.946549,1.400630,7.488938,0.225796,1.282029,0.172540,0.016765,4.334
9,1,2,3,0.367927,18.791877,7.117205,7.424419,7.996165,0.293319,0.712445,0.465821,0.012109,4.357
233,33,2,3,0.438172,21.570375,6.835835,4.623487,6.864687,0.556824,0.737687,0.394685,0.002322,4.604
121,17,2,3,0.397041,24.230399,8.618598,6.213632,7.090665,0.237832,0.546989,0.062370,0.004378,4.626
183,26,1,3,0.189443,20.204569,4.585410,9.979074,7.955353,0.399319,1.393149,0.362302,0.008128,4.950
274,39,1,3,0.338181,20.051824,7.819648,8.774394,7.399168,0.546375,1.193535,0.281767,0.003757,4.981
281,40,1,3,0.429360,20.720061,9.752715,8.581795,6.503783,0.438611,0.652446,0.292303,0.014313,5.093


In [12]:
df_LHS = pd.read_csv('data/estimation/LHS_sampling/data.csv')